In [1]:
from bs4 import BeautifulSoup
import urllib.request as request
import re

In [4]:
folder = r'C:/Users/bijaw/Desktop/scraped' + '/'

In [5]:
url = 'http://www.dermnet.com/images/Acne-Closed-Comedo'

In [6]:
response = request.urlopen(url)

In [7]:
soup = BeautifulSoup(response,'html.parser')

In [18]:
images = soup.find_all('img',{'src':re.compile('.jpg')})

In [19]:
for image in images: 
    print(image['src']+'\n')

/images/facebook.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-002.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-003.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-1.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-1.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-10.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-11.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-12.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-13.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-14.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-15.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-16.jpg

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-17.jpg

/images/facebook.jpg



In [3]:
import requests
from PIL import Image
from bs4 import BeautifulSoup
from io import BytesIO
import os
import shutil

root = 'http://www.dermnet.com'

'''type_name = ['Acne-and-Rosacea-Photos', 'Actinic-Keratosis-Basal-Cell-Carcinoma-and-other-Malignant-Lesions', 
             'Atopic-Dermatitis-Photos', 'Bullous-Disease-Photos', 'Cellulitis-Impetigo-and-other-Bacterial-Infections', 
             'Eczema-Photos', 'Exanthems-and-Drug-Eruptions', 'Hair-Loss-Photos-Alopecia-and-other-Hair-Diseases',
             'Herpes-HPV-and-other-STDs-Photos', 'Light-Diseases-and-Disorders-of-Pigmentation',
             'Lupus-and-other-Connective-Tissue-diseases', 'Melanoma-Skin-Cancer-Nevi-and-Moles', 'Nail-Fungus-and-other-Nail-Disease', 
             'Poison-Ivy-Photos-and-other-Contact-Dermatitis', 'Psoriasis-pictures-Lichen-Planus-and-related-diseases', 
             'Scabies-Lyme-Disease-and-other-Infestations-and-Bites', 'Seborrheic-Keratoses-and-other-Benign-Tumors', 
             'Systemic-Disease', 'Tinea-Ringworm-Candidiasis-and-other-Fungal-Infections', 
             'Urticaria-Hives', 'Vascular-Tumors', 'Vasculitis-Photos', 'Warts-Molluscum-and-other-Viral-Infections']'''
type_name = ['Seborrheic-Keratoses-and-other-Benign-Tumors']


type_LinksA = []
type_PagesA = []
type_SubLinksA = []

def get_max(link_):
    r_ = requests.get(link_)
    html_page1_1 = r_.text
    soup_page1_1 = BeautifulSoup(html_page1_1, "html.parser")

    navigationL = soup_page1_1.find_all("div", attrs={"class": "left"})
    max_ = 1

    if not navigationL:
        pass
    else:
        for navi_ in navigationL: 
            for i in navi_.children:
                try:
                    i_ = i.contents[0]
                except:
                    pass
                else:
                    if i_ == 'Next':
                        max_ = int(last_)
                        break
                    else: 
                        last_ = i_
    return int(max_)

'''
    Util: /Photo/ -> Real Links
'''
def photo2links(PhotosL):
    thumbRLinks = []
    
    for url_ in PhotosL:
        # print url_ 
        soup_page1_1_ = BeautifulSoup(requests.get(url_).text, "html.parser")

        for link in soup_page1_1_.find_all("img"):
            link_ = link.get("src")
            if 'Thumb' in link_:
                # print link_
                thumbRLinks.append(link_.replace('Thumb', ''))
        # print "\n"
    return list(set(thumbRLinks))

def download(url, image_path):
    try:
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(image_path, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
    except Exception as e:
        print(e)
        print("Failed to save " + image_path)
        print(url + "\n")
        # print image_path

    else:
        print("Successfully saved " + image_path)

'''
    Main
'''
def main():
    for sub_ in type_name:
        SubLinks = []
        # SubLinks = {}
        
        r_ = requests.get(root + '/images/' + sub_)
        html_page1 = r_.text
        soup_page1 = BeautifulSoup(html_page1, "html.parser")

        for link in soup_page1.find_all('a'):
            id_ = link.get('href')
            if '/images/' in id_:
                # SubLinks[root + id_] = "."
                SubLinks.append(root + id_)
        type_LinksA.append(SubLinks)

    attr = '/photos/'
    #print(type_LinksA[0])
    for type_ in type_LinksA:
        maxL_ = []
        thumbRLinks = []
        #print(type_)
        for link_ in type_:
            PhotosL = []
            max_ = get_max(link_)
            maxL_.append(max_)
            print(link_)
            for i_ in range(max_):
                PhotosL.append(link_ + attr + str(i_+1))
            
            print ("Pages: ", len(PhotosL))
            realLinks = photo2links(PhotosL)
            print("Links: ", len(realLinks))
            thumbRLinks.append(realLinks)

        type_PagesA.append(maxL_)
        type_SubLinksA.append(thumbRLinks)

    dir_root = 'C:/Users/bijaw/Desktop/scraped/'

    if os.path.exists(dir_root):
        pass    
    else:
        os.mkdir(dir_root) 
    for category in type_name:
        iloc = type_name.index(category)
        dir_disease = dir_root + category + '/'
        
        if os.path.exists(dir_disease):
            pass    
        else:
            os.mkdir(dir_disease) 
            
        print("Disease: ", dir_disease)
        for sub_ in type_LinksA[iloc]:
            iloc_sub = type_LinksA[iloc].index(sub_)
            # print iloc_sub
            # print sub_.split('/')[-1]
            dir_sub_disease = dir_disease + sub_.split('/')[-1]
            # print dir_sub_disease
            
            if os.path.exists(dir_sub_disease):
                pass    
            else:
                os.mkdir(dir_sub_disease)
            
            for l_ in type_SubLinksA[iloc][iloc_sub]:
                img_path = dir_sub_disease + '/' + l_.split('/')[-1]
                img_url = l_
                download(img_url, img_path)

if __name__ == "__main__":
    main()

http://www.dermnet.com/images/Accessory-Nipple
Pages:  1
Links:  10
http://www.dermnet.com/images/Accessory-Trachus
Pages:  1
Links:  6
http://www.dermnet.com/images/Acquired-Digital-Fibrokeratoma
Pages:  1
Links:  7
http://www.dermnet.com/images/Atypical-Fibroxanthoma
Pages:  1
Links:  2
http://www.dermnet.com/images/Calcifying-Epithelioma
Pages:  1
Links:  5
http://www.dermnet.com/images/Chondrodermatitis-Nodularis
Pages:  1
Links:  12
http://www.dermnet.com/images/Collagenoma
Pages:  1
Links:  1
http://www.dermnet.com/images/Connective-Tissue-Nevus
Pages:  1
Links:  2
http://www.dermnet.com/images/Cylindroma
Pages:  1
Links:  12
http://www.dermnet.com/images/Dermatofibroma
Pages:  1
Links:  12
http://www.dermnet.com/images/Dermatosis-Papulosa-Nigra
Pages:  1
Links:  8
http://www.dermnet.com/images/Eccrine-Spiradenoma
Pages:  1
Links:  3
http://www.dermnet.com/images/Endometriosis
Pages:  1
Links:  2
http://www.dermnet.com/images/Epidermal-Cyst
Pages:  1
Links:  12
http://www.dermnet

Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Chondrodermatitis-Nodularis/chondrodermatitis-nodularis-11.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Chondrodermatitis-Nodularis/chondrodermatitis-nodularis-16.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Chondrodermatitis-Nodularis/chondrodermatitis-nodularis-1.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Chondrodermatitis-Nodularis/chondrodermatitis-nodularis-17.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Chondrodermatitis-Nodularis/chondrodermatitis-nodularis-19.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Chondrodermatitis-Nodularis/chondrodermatitis-nodularis-13.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seb

Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Cyst/07GiantComedone.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Cyst/20EpidermCystRuptured.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Cyst/20PilarCyst1.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Cyst/20PilarCyst0825.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Nevus/epidermal-nevus-2.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Nevus/epidermal-nevus-16.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Epidermal-Nevus/epidermal-nevus-1.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-ot

Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Intraepidermal-Epithelioma/intraepidermal-epithelioma-1.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-18.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-14.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-17.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-12.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-1.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-2.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Keloids/keloids-16.jpg
Successfully saved C:

Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Melanoma-Mimic/melanoma-mimic-12.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Neurofibromas/neurofibromas-3.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Neurofibromas/neurofibromas-16.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Neurofibromas/neurofibromas-1.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Neurofibromas/neurofibromas-12.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Neurofibromas/neurofibromas-11.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Neurofibromas/neurofibromas-13.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tu

Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Sebaceous-Hyperplasia/sebaceous-hyperplasia-103.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Sebaceous-Hyperplasia/sebaceous-hyperplasia-13.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Seborrheic-Keratoses-Ruff/seborrheic-keratoses-ruff-104.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Seborrheic-Keratoses-Ruff/seborrheic-keratoses-ruff-107.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Seborrheic-Keratoses-Ruff/seborrheic-keratoses-ruff-102.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Seborrheic-Keratoses-Ruff/seborrheic-keratoses-ruff-100.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign

Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Spiny-keratoderma/spiny-keratoderma-5.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Spiny-keratoderma/spiny-keratoderma-4.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Stucco-Keratoses/stucco-keratoses-11.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Stucco-Keratoses/stucco-keratoses-16.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Stucco-Keratoses/stucco-keratoses-15.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Stucco-Keratoses/stucco-keratoses-13.jpg
Successfully saved C:/Users/bijaw/Desktop/scraped/Seborrheic-Keratoses-and-other-Benign-Tumors/Stucco-Keratoses/stucco-keratoses-18.jpg
Successfully saved C:/Users/bijaw/Desktop/scra